In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [3]:
#Match missing works to their composers before concatenating.
import pandas as pd
import Levenshtein

w_dict={}

cursor = conn.cursor()  

query = "SELECT name, uri, aka FROM (" + \
        "  SELECT name, uri, aka, counter, row_number() OVER (PARTITION BY name ORDER BY counter DESC ) AS num" + \
        "  FROM (" + \
        "         SELECT lower(w.name) as name, w.uri, w.aka, count(*) AS counter" + \
        "         FROM works w" + \
        "         LEFT JOIN listing_mappers lm ON w.uri = lm.work_uri" + \
        "         LEFT JOIN listings l ON l.composer = lm.composer AND l.work = lm.work AND l.source_id = lm.source_id" + \
        "         GROUP BY lower(w.name), w.uri, w.aka" + \
        "       ) a" + \
        ") b WHERE num = 1"

cursor.execute(query)

w_df = pd.DataFrame(cursor.fetchall(), columns=['name', 'uri', 'aka'])

for idx,row in w_df.iterrows():
    w_dict[row['name']]=row['uri']
    
works = list(w_dict.values())  

query = "SELECT lower(l.work), SUM(CASE WHEN l.source_id IN (6, 11) THEN CAST(l.additional_text AS INT) ELSE 1 END) " + \
        "FROM listings l LEFT JOIN listing_mappers lm ON l.composer = lm.composer " + \
        "AND l.work = lm.work AND l.source_id = lm.source_id WHERE lm.source_id IS NULL AND l.source_id = 6 " + \
        "GROUP BY lower(l.work) ORDER BY 2 desc"

cursor = conn.cursor()          
cursor.execute(query)
l_df = pd.DataFrame(cursor.fetchall(), columns=['work', 'counter'])

In [4]:
#mapping based on work's aka
map_dict={}
# print(l_df.shape)

for idx,row in l_df.iterrows():

    if idx%250==0:
        print(idx)
        
    for idx2,w in w_df.iterrows():        
#         print(w['aka'])
        for ak in w['aka'].split('; '):
            dist_to_work = Levenshtein.distance(row['work'], ak.lower())
            if dist_to_work <= 2:
                if map_dict.get(row['work'].lower()):
                    distance = map_dict[row['work'].lower()][1]
                    if dist_to_work < distance:
                        map_dict[row['work'].lower()]=[ak.lower(), dist_to_work, w['uri']]
                else:
                    map_dict[row['work'].lower()]=[ak.lower(), dist_to_work, w['uri']]
            elif dist_to_work == 0:
                map_dict[row['work'].lower()]=[ak.lower(), dist_to_work, w['uri']]
                break
                
    #             print("List: " + row['work'] + " ---" + " work: " + w.lower() + " ---" + " distance: " + str(Levenshtein.distance(row['work'], w.lower())))        

0
250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750
4000
4250
4500
4750
5000
5250
5500
5750
6000
6250
6500
6750
7000
7250
7500
7750
8000
8250
8500
8750
9000


In [20]:
map_dict
# w_df

{'la bohëme': ['la bohème', 1, 'Q188621'],
 '': ['13', 2, 'Q3597401'],
 'cosï fan tutte': ['cosi fan tutte', 1, 'Q207410'],
 'cavalleria rusticana': ['cavalleria rusticana', 0, 'Q221806'],
 'trial by jury': ['trial by jury', 0, 'Q250903'],
 'la fille du règiment': ['la fille du régiment', 1, 'Q729568'],
 'die zauberflˆte': ['die zauberflöte', 1, 'Q5064'],
 'the fantasticks': ['the fantasticks', 0, 'Q4481121'],
 'trouble in tahiti': ['trouble in tahiti', 0, 'Q377876'],
 'die walk¸re': ['die walküre', 1, 'Q324319'],
 'la pèrichole': ['la perichole', 1, 'Q730336'],
 'a funny thing happened on the way to the forum': ['a funny thing happened on the way to the forum',
  0,
  'Q862709'],
 'romèo et juliette': ['romeo et juliette', 1, 'Q918714'],
 'guys and dolls': ['guys and dolls', 0, 'Q796988'],
 'a little night music': ['a little night music', 0, 'Q2530921'],
 'kismet': ['kismet', 0, 'Q3815652'],
 'mefistofele': ['mefistofele', 0, 'Q935669'],
 'west side story': ['west side story', 0, 'Q66

In [27]:
cursor = conn.cursor()

for key in list(map_dict.keys()): 
    if key!='':
#         print(key)
#         match = w_dict.get(map_dict.get(key)[0])
        insert = 'INSERT INTO LISTING_MAPPERS (SOURCE_ID, WORK, WORK_URI) VALUES ('
        values = "6, '" + str(key).replace("'", "''") + "', '" + str(map_dict.get(key)[2]) + "'); "
        commit = 'COMMIT;'    
#         print(insert + values + commit)
        cursor.execute(insert + values + commit)    

In [19]:
w_dict.get('jenufa')

In [ ]:
# w_dict

In [20]:
#mapping based on work's aka
map_dict={}

# w_df = w_df[w_df['uri']=='Q566892']
l_df = l_df[l_df['work']=='annie']

for idx,row in l_df.iterrows():        
    for idx2,w in w_df.iterrows():        
#         print(w['aka'])
        for ak in w['aka'].split('; '):
            dist_to_work = Levenshtein.distance(row['work'], ak.lower())
            if dist_to_work <= 2:
                print([ak.lower(), dist_to_work, w['uri']])
                if map_dict.get(row['work'].lower()):
                    distance = map_dict[row['work'].lower()][1]
                    if dist_to_work < distance:
                        map_dict[row['work'].lower()]=[ak.lower(), dist_to_work, w['uri']]
                else:
                    map_dict[row['work'].lower()]=[ak.lower(), dist_to_work, w['uri']]
#             print("List: " + row['work'] + " ---" + " work: " + w.lower() + " ---" + " distance: " + str(Levenshtein.distance(row['work'], w.lower())))        
map_dict

['agni', 2, 'Q4693190']
['ani', 2, 'Q19570835']
['annie', 0, 'Q566892']
['annie', 0, 'Q566892']
['jennie', 2, 'Q6177933']


{'annie': ['annie', 0, 'Q566892']}